In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df_advertising = pd.read_csv(r'C:\Users\tranv\Desktop\Data science\files\advertising.csv')